In [22]:
from python_simulation_manager.experiment_base.experiment_handler import ExperimentHandler, ExperimentOutput
from typing import override
import numpy as np

In [23]:
class MyTestOutput(ExperimentOutput):
    def __init__(self, file_name):
        super().__init__(file_name)
        self.first_line      = None
        self.sums            = None
        self.multiplications = None

    @override
    def parse_output(self, line_number, lines):
        if line_number == 0:
            self.first_line = lines
        else:
            slines = lines.split(", ")
            self.sums            = float(slines[0])
            self.multiplications = float(slines[1])

class MyTestExperiment(ExperimentHandler):

    @override
    def set_result_type(self, output_file) -> MyTestOutput:
        return MyTestOutput(output_file)
                
class TestExperimentBuilder:
    def __init__(self, name: str, folder: str, rust_dir: str):
        self.name: str     = name
        self.folder: str   = folder
        self.rust_dir: str = rust_dir

    def build_experiment(self: str, my_array: np.array, my_bool: bool, my_dict: dict):
     
        new_exp = MyTestExperiment(self.name, self.folder, self.rust_dir)
        new_exp.add_static_parameter("my_array", my_array)
        new_exp.add_static_parameter("my_bool", my_bool)
        new_exp.add_scaling_parameter("my_dict", my_dict)
        new_exp.set_scaling_name(["Lx", "Ly"])
        return new_exp

In [24]:
test_builder = TestExperimentBuilder(name="test_experiment", folder="test_results", rust_dir="rust_example")
an_array     = np.arange(0, 10, 1)
a_boolean    = True
a_dict       = {
    2: "small",
    4: "medium",
    8: "Large"
}

test_exp = test_builder.build_experiment(an_array, a_boolean, a_dict) 
test_exp.set_static_parameter("my_bool", False)

if not test_exp.are_parameter_files_available():
    test_exp.write_parameter_files()

>> Directory "rust_example/test_results/test_experiment" created
>> Succesfully added my_array
>> Succesfully added my_bool
>> Succesfully added my_dict
>> Setting file paths:
-- setting paramfile "test_results/test_experiment/parameter_2x2.txt"
-- setting paramfile "test_results/test_experiment/parameter_4x4.txt"
-- setting paramfile "test_results/test_experiment/parameter_8x8.txt"
-- setting outfile "test_results/test_experiment/out_2x2.txt"
-- setting outfile "test_results/test_experiment/out_4x4.txt"
-- setting outfile "test_results/test_experiment/out_8x8.txt"
>> "my_bool" set
>> Looking for parameter files:
* Not all parameter files created yet:
-- not written yet: rust_example/test_results/test_experiment/parameter_2x2.txt
-- not written yet: rust_example/test_results/test_experiment/parameter_4x4.txt
-- not written yet: rust_example/test_results/test_experiment/parameter_8x8.txt
>> writing parameter files:
-- test_results/test_experiment/parameter_2x2.txt: done
-- test_results/

In [29]:
for L in test_exp.get_lengths():
    if not test_exp.has_output(L):
        test_exp.perform_rust_computation(L)
        print("")

Command: "cargo run --release -- test_results/test_experiment/parameter_2x2.txt"
 * From Rust:     Finished ]8;;https://doc.rust-lang.org/cargo/reference/profiles.html#default-profiles\`release` profile [optimized]]8;;\ target(s) in 0.02s
 * From Rust:      Running `target\release\rust_example.exe test_results/test_experiment/parameter_2x2.txt`
 * From Rust: Reading parameters:
 * From Rust: Lx: 2
 * From Rust: Ly: 2
 * From Rust: my_array: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
 * From Rust: my_bool: False
 * From Rust: my_dict: small
 * From Rust: outputfile: test_results/test_experiment/out_2x2.txt
 * From Rust: Writing in test_results/test_experiment/out_2x2.txt

Command: "cargo run --release -- test_results/test_experiment/parameter_4x4.txt"
 * From Rust:     Finished ]8;;https://doc.rust-lang.org/cargo/reference/profiles.html#default-profiles\`release` profile [optimized]]8;;\ target(s) in 0.02s
 * From Rust:      Running `target\release\rust_example.exe test_results/test_experimen

In [27]:
test_exp.are_results_available()

>> Looking for output files:
-- Found output: "rust_example/test_results/test_experiment/out_2x2.txt"
-- Found output: "rust_example/test_results/test_experiment/out_4x4.txt"
-- Found output: "rust_example/test_results/test_experiment/out_8x8.txt"


True

In [28]:
results = test_exp.get_results()
print("")
for (Lx, result) in results.items():
    print(result.first_line, end='')
    print("sums:", result.sums)
    print("multiplications:", result.multiplications)


Found output: "rust_example/test_results/test_experiment/out_2x2.txt
Found output: "rust_example/test_results/test_experiment/out_4x4.txt
Found output: "rust_example/test_results/test_experiment/out_8x8.txt

Hello world! [Lx=2]
sums: 4.0
multiplications: 4.0
Hello world! [Lx=4]
sums: 8.0
multiplications: 16.0
Hello world! [Lx=8]
sums: 16.0
multiplications: 64.0
